In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import fsolve, minimize
from scipy.optimize import minimize, LinearConstraint

In [2]:
data_chi = pd.read_stata('C:\\Users\\chans\\Desktop\\UpdatedEmpirics\\cutoffs.dta')

In [5]:
x = data_chi['neg_avgcpi'].values
y_min = data_chi['max_china_cutoff'].values
y_max = data_chi['min_upper'].values

# Define objective function for boundary violations
def objective(coeffs):
    a, b = coeffs
    y_pred = a * x + b
    violations = np.sum((y_pred < y_min) | (y_pred > y_max))
    return violations

# Initial guess and constraints for optimization
initial_guess = [0, 1]
constraints = ({'type': 'ineq', 'fun': lambda coeffs: np.min(coeffs[0] * x + coeffs[1] - y_min)},
               {'type': 'ineq', 'fun': lambda coeffs: np.max(y_max - (coeffs[0] * x + coeffs[1]))})

# Find optimal A and B
result = minimize(objective, initial_guess, method='SLSQP', constraints=constraints)
a_best, b_best = result.x

# Apply transformation and check bounds
data_chi['chi_value'] = a_best * data_chi['neg_avgcpi'] + b_best

data_chi['chi_value'] = data_chi.apply(
    lambda row: min(max(row['chi_value'], row['max_china_cutoff'] + 0.05), row['min_upper'] - 0.05),
    axis=1
)


In [6]:
file_path = 'C:\\Users\\chans\\Desktop\\UpdatedEmpirics\\chi_values.dta'
data_chi.to_stata(file_path)